Set up environment and load in data

In [ ]:
library(tidyverse)
library(brms)
library(here)
library(lemon)
theme_set(theme_bw())
helpers_path = paste0(here(),'/analysis/helpers/')
inputs_path = paste0(here(),'/inputs/')
source(paste0(helpers_path, '01_clean_behavioral_data.R'))
rm(data_bc_clean)
fig_out_path = paste0(here(), '/outputs/fig/')

Add columns that will be used for plots below.

In [ ]:
data_yn_clean = data_yn_clean %>%
  mutate(correct = ifelse(possiblePayoff>reference & yesChosen == 1, 1, ifelse(possiblePayoff < reference & yesChosen == 0, 1, 0))) %>%
  mutate(type_chr = ifelse(type == 1, "HT", "RE"),
         week = ifelse(week == 1, "Week 1", ifelse(week == 2, "Week 2", ifelse(week == 3, "Week 3", NA))),
         week = factor(week, levels = c("Week 1", "Week 2", "Week 3"))) %>%
  mutate(val_diff = possiblePayoff - reference,
         abs_val_diff = abs(possiblePayoff - reference),
         val_diff_bin = round(val_diff/50),
         val_diff_bin_str = paste0(val_diff_bin*50-25,":",val_diff_bin*50+25),
         val_diff_bin_str = factor(val_diff_bin_str, levels = c("-225:-175", "-175:-125", "-125:-75", "-75:-25", "-25:25", "25:75", "75:125", "125:175", "175:225")))

# Definition with shape

Function to get stimnums and attribute levels for HT stims for a given subject

In [ ]:
get_sub_ht_stims = function(cur_sub){
  
  out = data_yn_clean %>%
    filter(subnum == cur_sub & type == 1) %>%
    select(type, stimNum, shape, filling, orientation) %>%
    distinct() %>%
    arrange(stimNum)
  
  return(out)
}


Test function

In [ ]:
get_sub_ht_stims(601)

Function to get stimnums and attribute levels for stims close to a given HT stim num for a given subject

In [ ]:
get_sub_cht_stims_for_one_ht_wshape = function(cur_sub, ht_stim){
  
  # For each of the HT stims define CHT stims as:
  # same filling, same orientation, different shape (5 stims)
  # same shape, same orientation. filling level +/- 1 (2 stims)
  # same shape, filling, orientation level +/1 (2 stims)
  # Exclude if any CHT is also an HT stim
  
  sub_ht_stim = data_yn_clean %>%
    filter(subnum == cur_sub & type == 1 & stimNum == ht_stim) %>%
    select(type, stimNum, shape, filling, orientation) %>%
    distinct()
  
  shape_level = sub_ht_stim$shape[1]
  orientation_level = sub_ht_stim$orientation[1]
  filling_level = sub_ht_stim$filling[1]
  
  # List all levels of all attributes
  shape_levels = sort(unique(data_yn_clean$shape))
  orientation_levels = sort(unique(data_yn_clean$orientation))
  filling_levels = sort(unique(data_yn_clean$filling))
  
  # Get close shape levels
  close_shape_levels = shape_levels[shape_levels != shape_level]
  out = tibble(shape = close_shape_levels, filling = filling_level, orientation = orientation_level)

  # Get close orientation levels
  orientation_level_index = which(orientation_levels == orientation_level)  
  
  if(orientation_level_index == 1){  # if o = 0, get o = 15, o = 150
    close_orientation_levels = c(orientation_levels[2], orientation_levels[length(orientation_levels)])
  } else if (orientation_level_index == length(orientation_levels)){ #if o = 150, get o = 135, o = 0
    close_orientation_levels = c(orientation_levels[1], orientation_levels[length(orientation_levels) - 1])  
  } else{
    close_orientation_levels = orientation_levels[c(orientation_level_index-1, orientation_level_index+1)]
  }
  
  out = rbind(out, tibble(shape = shape_level, filling = filling_level, orientation = close_orientation_levels))
  
  
  # Get close filling levels
  filling_level_index = which(filling_levels == filling_level)
  
  if(filling_level_index == 1){
    close_filling_levels = filling_levels[2]
  } else if(filling_level_index == length(filling_levels)){
    close_filling_levels = filling_levels[filling_level_index - 1]
  } else {
    close_filling_levels = filling_levels[c(filling_level_index - 1, filling_level_index + 1)]
  }
  
  out = rbind(out, tibble(shape = shape_level, filling = close_filling_levels, orientation = orientation_level))
  
  # Get stimNums for the CHT 
  out = data_yn_clean %>%
    filter(subnum == cur_sub) %>%
    select(stimNum, type, shape, filling, orientation) %>%
    right_join(out, by = c("shape", "filling", "orientation")) %>%
    arrange(stimNum) %>%
    
    # Check if any of the CHT are also HT
    filter(type != 1) %>%
    distinct()
  
  return(out)
}

Test function

In [ ]:
get_sub_cht_stims_for_one_ht_wshape(601, 85)

In [ ]:
get_sub_cht_stims_for_one_ht_wshape(601, 670)

In [ ]:
get_sub_cht_stims_for_one_ht_wshape(601, 332)

Wrapper to get all close to HT stims for a given subject

In [ ]:
get_sub_cht_stims_for_all_ht_wshape = function(cur_sub){
  
  sub_ht_stims = get_sub_ht_stims(cur_sub)
  
  out = tibble()
  
  for(cur_ht_stim in sub_ht_stims$stimNum){
    out = rbind(out, get_sub_cht_stims_for_one_ht_wshape(cur_sub, cur_ht_stim))
  }
  
  return(out)
  
}

In [ ]:
get_sub_cht_stims_for_all_ht_wshape(601)

## Assign CHT labels to dataset

In [ ]:

out = tibble()
for(cur_sub in unique(data_yn_clean$subnum)){
  sub_cht_stimnums = get_sub_cht_stims_for_all_ht_wshape(cur_sub)
  
  sub_cht_stimnums = sub_cht_stimnums$stimNum
  
  cur_out = data_yn_clean %>%
    filter(subnum == cur_sub) %>%
    mutate(type_chr = ifelse(stimNum %in% sub_cht_stimnums, "CHT", type_chr))
  
  out = rbind(out, cur_out)
}

data_yn_cht_wshape = out
rm(out, sub_cht_stimnums, cur_out)


In [ ]:
with(data_yn_cht_wshape, table(subnum, type_chr))

## Accuracy for CHT

In [ ]:
sub_summary = data_yn_clean %>%
  group_by(day, subnum, type_chr) %>%
  summarise(mean_accuracy = mean(correct),
            sem_accuracy = sd(correct)/sqrt(n()),
            .groups = 'keep') %>%
  mutate(task = "Y/N (two stim categories)",
         sum_type = "sub",
         type_chr = factor(type_chr, levels = c("HT", "RE", "CHT"))) %>%
  rbind(data_yn_cht_wshape %>%
  group_by(day, subnum, type_chr) %>%
  summarise(mean_accuracy = mean(correct),
            sem_accuracy = sd(correct)/sqrt(n()),
            .groups = 'keep') %>%
  mutate(task = "Y/N (three stim categories w/ shape)",
         sum_type = "sub",
         type_chr = factor(type_chr, levels = c("HT", "RE", "CHT")))) %>%
  mutate(task = factor(task, levels = c("Y/N (two stim categories)", "Y/N (three stim categories w/ shape)")))

group_summary = data_yn_clean %>%
  group_by(day, type_chr) %>%
  summarise(mean_accuracy = mean(correct),
            .groups = 'keep') %>%
  mutate(task = "Y/N (two stim categories)",
         sum_type = "group", subnum = 600,
         type_chr = factor(type_chr, levels = c("HT", "RE", "CHT"))) %>%
  rbind(data_yn_cht_wshape %>%
          group_by(day, type_chr) %>%
          summarise(mean_accuracy = mean(correct),
                    .groups = 'keep') %>%
          mutate(task = "Y/N (three stim categories w/ shape)",
                 sum_type = "group", subnum = 600,
                 type_chr = factor(type_chr, levels = c("HT", "RE", "CHT")))) %>%
  mutate(task = factor(task, levels = c("Y/N (two stim categories)", "Y/N (three stim categories w/ shape)")))

In [ ]:
p = ggplot()+
  geom_point(data = group_summary, aes(day, mean_accuracy, color = type_chr))+

  geom_line(data = group_summary, aes(day, mean_accuracy, color = type_chr))+
  
  geom_line(data = sub_summary, aes(day, mean_accuracy, color = type_chr, linetype = as.factor(subnum)), alpha = .3)+

  facet_wrap(~task)+
  labs(x = "Day", y = "Accuracy", color="", title = "Three stim types (w/ shape)")+
  theme(panel.grid.minor.x = element_blank(),
        panel.grid.major.x = element_blank(),
        legend.position = "bottom",
        legend.box.margin=margin(t = -10),
        legend.spacing.y = unit(-0.25, "cm"),
        legend.box="vertical")+
  scale_x_continuous(breaks=seq(1,11,1))+
  scale_y_continuous(breaks=seq(.7,1,.1), limits = c(.7, 1))+
  scale_color_brewer(palette = "Dark2")+
  scale_linetype_manual(values = rep("solid", 6))+
  guides(linetype = "none")+
  annotate("rect", xmin = 2.75, xmax = 3.25, ymin = .7, ymax = 1,
           alpha = .25,fill = "gray")+
  annotate("rect", xmin = 6.75, xmax = 7.25, ymin = .7, ymax = 1,
           alpha = .25,fill = "gray")+
  annotate("rect", xmin = 10.75, xmax = 11.25, ymin = .7, ymax = 1,
           alpha = .25,fill = "gray")

# ggsave(file=paste0(fig_out_path, 'yn_groupAndSub_accuracyOverDaysCHT.jpg'), p, height = 5, width=12, units="in")
p

In [ ]:
p = sub_summary %>%
  filter(task =="Y/N (three stim categories w/ shape)") %>%
  ggplot(aes(day, mean_accuracy, color = type_chr))+
  geom_point(position = position_dodge(width=.25))+
  geom_line(alpha =.5)+
  geom_errorbar(aes(ymin = mean_accuracy - sem_accuracy, ymax = mean_accuracy + sem_accuracy),position = position_dodge(width=.25), width = .2)+
  facet_wrap(~subnum)+
  labs(x = "Day", y = "Accuracy", color="", title = "Three stim types (w/ shape)")+
  theme(panel.grid.minor.x = element_blank(),
        panel.grid.major.x = element_blank(),
        legend.position = "bottom",
        legend.box.margin=margin(t = -10),
        legend.spacing.y = unit(-0.25, "cm"),
        legend.box="vertical")+
  scale_x_continuous(breaks=seq(1,11,1))+
  scale_y_continuous(breaks=seq(.7,1,.1), limits = c(.6, 1))+
  scale_color_brewer(palette = "Dark2")+
 
  annotate("rect", xmin = 2.75, xmax = 3.25, ymin = .6, ymax = 1,
           alpha = .25,fill = "gray")+
  annotate("rect", xmin = 6.75, xmax = 7.25, ymin = .6, ymax = 1,
           alpha = .25,fill = "gray")+
  annotate("rect", xmin = 10.75, xmax = 11.25, ymin = .6, ymax = 1,
           alpha = .25,fill = "gray")

# ggsave(file=paste0(fig_out_path, 'yn_sub_accuracyOverDaysCHT.jpg'), p, height = 8, width=16, units="in")
p

## Model for accuracy difference

Forward difference coding of stim type comparing
RE to CHT
CHT to HT
So level 1: RE, level 2: CHT, level 3: HT

In [ ]:
contrasts(as.factor(data_yn_cht_wshape$type_chr))

In [ ]:
contrasts(factor(data_yn_cht_wshape$type_chr, levels = c("RE", "CHT", "HT")))

In [ ]:
k = 3

my_fwd_diff = matrix(c(2/k, -1/k, -1/k,
                       1/k, 1/k, -2/k), ncol = k-1)

my_fwd_diff

Run model for each subject with the forward difference coding for type

In [ ]:
fn_draws = paste0(inputs_path, 'yn_correct_cht_wshape_fwd_diff_draws.csv')

if(file.exists(fn_draws)){
  yn_acc_draws = read.csv(fn_draws)
} else{
  subnums = unique(data_yn_cht_wshape$subnum)

  normMax = 1
  normMin = 0

  yn_acc_draws = tibble()

  for(cur_sub in subnums){
    
    cur_dat = data_yn_cht_wshape %>% 
      ungroup() %>%
      filter(subnum == cur_sub) %>%
      mutate(type_chr = factor(type_chr, levels = c("RE", "CHT", "HT")),
             norm_day = (normMax - normMin) / (max(day) - min(day)) * (day - max(day)) + (normMax))
    
    contrasts(cur_dat$type_chr) = my_fwd_diff
    
    cur_m <- brm(formula = correct ~  norm_day * type_chr,
                        data= cur_dat,
                        family = bernoulli(link = "logit"),
                        warmup = 500,
                        iter = 2500,
                        chains = 4,
                        init = "0",
                        cores = 4,
                        seed = 389253)

    cur_draws = as_draws_df(cur_m)
    cur_draws$subnum = cur_sub

    yn_acc_draws = rbind(yn_acc_draws, cur_draws)

  }
  names(yn_acc_draws) = gsub(":", ".", names(yn_acc_draws))
  write.csv(yn_acc_draws, fn_draws, row.names = F)

  rm(cur_dat, cur_m, cur_draws, cur_sub)
}

In [ ]:
yn_par_summary = as_tibble(yn_acc_draws) %>%
  select(b_Intercept, b_norm_day, b_type_chr1, b_type_chr2, b_norm_day.type_chr1, b_norm_day.type_chr2, subnum) %>%
  gather(key, value, -subnum) %>%
  mutate(key=recode(key, 
                    "b_Intercept" = "Intercept", 
                    "b_norm_day" = "Day [0, 1]",
                    "b_type_chr1" = "Type (RE vs CHT)",
                    "b_type_chr2" = "Type (CHT vs HT)",
                    "b_norm_day.type_chr1" = "Day x Type (RE vs CHT)",
                    "b_norm_day.type_chr2" = "Day x Type (CHT vs HT)"),
         key = factor(key, levels = c("Intercept", "Day [0, 1]", "Type (RE vs CHT)", "Type (CHT vs HT)", "Day x Type (RE vs CHT)", "Day x Type (CHT vs HT)"))) %>%
  group_by(subnum, key) %>%
  summarise(.groups = 'keep',
            m = mean(value),
            l95 = HDInterval::hdi(value)[1],
            h95 = HDInterval::hdi(value)[2]) %>%
  mutate(subnum = as.factor(subnum),
         task = "Y/N") 

p = yn_par_summary %>%
  ggplot(aes(m, subnum)) +
  geom_vline(aes(xintercept = 0), color = "gray") + 
  geom_point(position = position_dodge(width=.75))+
  geom_errorbarh(aes(xmin = l95, xmax = h95, height=.2), position = position_dodge(width=.75))+
  facet_grid(.~key, scales = 'free_x') +
  labs(x = "Parameter Estimate", y = "Subject", color = "", title = "95% HDI for posterior slopes - Correct ~ Day x Type (3 levels; with shape)") +
  theme(legend.position = "bottom",
        panel.grid = element_blank(),
        legend.box.margin=margin(t = -10))+
  scale_x_symmetric()

# ggsave(file=paste0(fig_out_path, 'yn_accCHTwshape_ParEsts.jpg'), p, height = 3, width=12, units="in")

p

## Psychometric plots for CHT

In [ ]:
sub_summary = data_yn_cht_wshape %>%
  mutate(time_bin = ifelse(day == 1, "Day 1", ifelse(day == 2, "Day 2", ifelse(day == 3, "Day 3", ifelse((day > 3) & (day < 8), "Week 2", "Week 3"))))) %>%
  group_by(subnum, time_bin, val_diff_bin_str, type_chr) %>%
  summarise(mean_y = mean(yesChosen),
            sem_y = sd(yesChosen)/sqrt(n()),
            .groups="keep") %>%
  mutate(type_chr = factor(type_chr, levels = c("HT", "CHT", "RE")))

group_summary = data_yn_cht_wshape %>%
  mutate(time_bin = ifelse(day == 1, "Day 1", ifelse(day == 2, "Day 2", ifelse(day == 3, "Day 3", ifelse((day > 3) & (day < 8), "Week 2", "Week 3"))))) %>%
  group_by(time_bin, val_diff_bin_str, type_chr) %>%
  summarise(mean_y = mean(yesChosen),
            .groups="keep")%>%
    mutate(type_chr = factor(type_chr, levels = c("HT", "CHT", "RE")))

In [ ]:
p = ggplot() +
  geom_hline(data = group_summary, aes(yintercept =.5), color="gray")+
  geom_vline(data = group_summary, aes(xintercept = 5), color="gray")+
  geom_point(data = group_summary, aes(val_diff_bin_str, mean_y, color = type_chr, alpha = time_bin, size = time_bin))+
  geom_line(data = group_summary, aes(val_diff_bin_str, mean_y, color = type_chr, group = time_bin, alpha = time_bin)) +
  facet_wrap(~type_chr)+
  # labs(x = "Value Stim - Value Reference", y = "p(Yes)", title="Y/N Task", alpha="", size = "")+
    labs(x = "Value Stim - Value Reference", y = "p(Yes)", alpha="", size = "", title = "Y/N Task (3 levels with shape)")+
  theme(panel.grid = element_blank(),
        legend.position = "bottom",
        axis.text.x = element_text(angle = 15, margin = margin(t = 5), size = 14),
        axis.text.y = element_text(size = 14),
        strip.text = element_text(size = 14),
        legend.box.margin=margin(t = -10),
        legend.spacing.y = unit(-0.25, "cm"),
        legend.box="vertical",
        legend.text = element_text(size=14),
        axis.title = element_text(size = 16))+
  # scale_color_brewer(palette = "Dark2")+
  scale_color_manual(values = c("#1B9E77", "#7570B3", "#D95F02" ))+
  scale_x_discrete(labels=c("-225:-175", "", "-125:-75", "", "-25:25", "", "75:125", "", "175:225"))+
  scale_y_continuous(breaks = seq(0,1,.5))+
  guides(linetype = "none", color = "none")+
  scale_alpha_manual(values = c(1, .6, .5, .3, .2))+
  scale_size_manual(values = c(1.5, 2 , 2.5, 3, 3.5))

# ggsave(file=paste0(fig_out_path, 'yn_group_pYesOverValDiffAcrossTimeCHTwShape.jpg'), p, height = 4, width = 12, units="in")

p

In [ ]:
p = ggplot() +
  geom_hline(data = sub_summary, aes(yintercept =.5), color="gray")+
  geom_vline(data = sub_summary, aes(xintercept = 5), color="gray")+
  geom_point(data = sub_summary, aes(val_diff_bin_str, mean_y, color = type_chr, alpha = time_bin, size = time_bin))+
  geom_line(data = sub_summary, aes(val_diff_bin_str, mean_y, color = type_chr, group = time_bin, alpha = time_bin)) +
  geom_errorbar(data = sub_summary, aes(x = val_diff_bin_str, ymin = mean_y-sem_y, ymax = mean_y + sem_y, color = type_chr), width = 0, alpha = .6)+
  facet_grid(type_chr~subnum)+
  labs(x = "Value Stim - Value Reference", y = "p(Yes)", title="Y/N Task (3 levels with shape)", alpha="", size = "")+
  theme(panel.grid = element_blank(),
        legend.position = "bottom",
        axis.text.x = element_text(angle = 15, margin = margin(t = 5)),
        legend.box.margin=margin(t = -10),
        legend.spacing.y = unit(-0.25, "cm"),
        legend.box="vertical")+
  scale_color_manual(values = c("#1B9E77", "#7570B3", "#D95F02" ))+
  scale_x_discrete(labels=c("-225:-175", "", "-125:-75", "", "-25:25", "", "75:125", "", "175:225"))+
  scale_y_continuous(breaks = seq(0,1,.5))+
  guides(linetype = "none", color = "none")+
  scale_alpha_manual(values = c(1, .6, .5, .3, .2))+
  scale_size_manual(values = c(1.5, 2 , 2.5, 3, 3.5))

# ggsave(file=paste0(fig_out_path, 'yn_subj_pYesOverValDiffAcrossTimeCHTwShape.jpg'), p, height = 7, width = 12, units="in")
p

## Logits for CHT

In [ ]:
k = 3

my_fwd_diff = matrix(c(2/k, -1/k, -1/k,
                       1/k, 1/k, -2/k), ncol = k-1)

my_fwd_diff

In [ ]:
fn_draws = paste0(inputs_path, 'yn_logit_norm_cht_wshape_draws.csv')

if(file.exists(fn_draws)){
  yn_logit_draws = read.csv(fn_draws)
} else{
  normMax = 1
  normMinDay = 0
  normMinVal = -1
  
  yn_logit_draws = tibble()
  
  for(cur_sub in unique(data_yn_clean$subnum)){
  
    cur_dat = data_yn_cht_wshape %>% 
      ungroup() %>%
      filter(subnum == cur_sub) %>%
      mutate(type_chr = factor(type_chr, levels = c("RE", "CHT", "HT")),
             norm_day = (normMax - normMinDay) / (max(day) - min(day)) * (day - max(day)) + (normMax)) %>%
      group_by(day) %>% # Grouping by day to normalize valdiff matters. This makes sure every day ranges [-1,1]
      mutate(norm_valDiff = (normMax - normMinVal) / (max(val_diff) - min(val_diff)) * (val_diff - max(val_diff)) + (normMax)) %>%
      select(yesChosen, norm_valDiff, norm_day, type_chr)
    
    contrasts(cur_dat$type_chr) = my_fwd_diff
    
    cur_m <- brm(formula = yesChosen ~ norm_valDiff * norm_day * type_chr,
                        data= cur_dat,
                        family = bernoulli(link = "logit"),
                        warmup = 500,
                        iter = 2500,
                        chains = 4,
                        init = "0",
                        cores = 4,
                        seed = 389253)
    
    cur_draws = as_draws_df(cur_m)
    cur_draws$subnum = cur_sub
    
    yn_logit_draws = rbind(yn_logit_draws, cur_draws)
  }
  names(yn_logit_draws) = gsub(":", ".", names(yn_logit_draws))
  write.csv(yn_logit_draws, fn_draws, row.names = F)

  rm(cur_dat, cur_m, cur_draws, cur_sub)
}

In [ ]:
names(as_tibble(yn_logit_draws))

In [ ]:
yn_par_summary = as_tibble(yn_logit_draws) %>%
  select(b_Intercept, b_norm_day, b_norm_valDiff, b_type_chr1, b_type_chr2, b_norm_valDiff.norm_day, b_norm_valDiff.type_chr1, b_norm_valDiff.type_chr2, b_norm_day.type_chr1, b_norm_day.type_chr2, b_norm_valDiff.norm_day.type_chr1, b_norm_valDiff.norm_day.type_chr2, subnum) %>%
  gather(key, value, -subnum) %>%
  mutate(key=recode(key, 
                    "b_Intercept" = "Intercept", 
                    "b_norm_day" = "Day [0, 1]",
                    "b_norm_valDiff" = "Value Diff [-1, 1]",
                    "b_type_chr1" = "Type (RE vs CHT)",
                    "b_type_chr2" = "Type (CHT vs HT)",
                    "b_norm_valDiff.norm_day" = "Value Diff x Day",
                    "b_norm_valDiff.type_chr1" = "Value Diff x Type (RE vs CHT)",
                    "b_norm_valDiff.type_chr2" = "Value Diff x Type (CHT vs HT)",
                    "b_norm_day.type_chr1" = "Day x Type (RE vs CHT)",
                    "b_norm_day.type_chr2" = "Day x Type (CHT vs HT)",
                    "b_norm_valDiff.norm_day.type_chr1" = "Value Diff x Day x Type (RE vs CHT)",
                    "b_norm_valDiff.norm_day.type_chr2" = "Value Diff x Day x Type (CHT vs HT)"),
         key = factor(key, levels = c("Intercept", "Day [0, 1]", "Value Diff [-1, 1]", "Type (RE vs CHT)", "Type (CHT vs HT)", "Value Diff x Day", "Value Diff x Type (RE vs CHT)", "Value Diff x Type (CHT vs HT)", "Day x Type (RE vs CHT)", "Day x Type (CHT vs HT)", "Value Diff x Day x Type (RE vs CHT)", "Value Diff x Day x Type (CHT vs HT)"))) %>%
  group_by(subnum, key) %>%
  summarise(.groups = 'keep',
            m = mean(value),
            l95 = HDInterval::hdi(value)[1],
            h95 = HDInterval::hdi(value)[2]) %>%
  mutate(subnum = as.factor(subnum),
         task = "Y/N") 

p = yn_par_summary %>%
  ggplot(aes(m, subnum)) +
  geom_vline(aes(xintercept = 0), color = "gray") + 
  geom_point(position = position_dodge(width=.75))+
  geom_errorbarh(aes(xmin = l95, xmax = h95, height=.2), position = position_dodge(width=.75))+
  facet_grid(.~key, scales = 'free_x', labeller = labeller(key = label_wrap_gen(10))) +
  labs(x = "Parameter Estimate", y = "Subject", color = "", title = "95% HDI for posterior slopes - Yes ~ Value Diff x Day x Type (3 levels; with shape)") +
  theme(legend.position = "bottom",
        panel.grid = element_blank(),
        legend.box.margin=margin(t = -10))+
  scale_x_symmetric()

# ggsave(file=paste0(fig_out_path, 'yn_logitCHTwshape_ParEsts.jpg'), p, height = 3, width=12, units="in")

p

## RT over days

In [ ]:
sub_summary = data_yn_clean %>%
  group_by(day, subnum, type_chr) %>%
  summarise(mean_y = mean(log(rt)),
            sem_y = sd(log(rt))/sqrt(n()),
            .groups = 'keep') %>%
  mutate(task = "Y/N (two stim categories)",
         sum_type = "sub",
         type_chr = factor(type_chr, levels = c("HT", "RE", "CHT"))) %>%
  rbind(data_yn_cht_wshape %>%
  group_by(day, subnum, type_chr) %>%
  summarise(mean_y = mean(log(rt)),
            sem_y = sd(log(rt))/sqrt(n()),
            .groups = 'keep') %>%
  mutate(task = "Y/N (three stim categories w/ shape)",
         sum_type = "sub",
         type_chr = factor(type_chr, levels = c("HT", "RE", "CHT")))) %>%
  mutate(task = factor(task, levels = c("Y/N (two stim categories)", "Y/N (three stim categories w/ shape)")))

group_summary = data_yn_clean %>%
  group_by(day, type_chr) %>%
  summarise(mean_y = mean(log(rt)),
            .groups = 'keep') %>%
  mutate(task = "Y/N (two stim categories)",
         sum_type = "group", subnum = 600,
         type_chr = factor(type_chr, levels = c("HT", "RE", "CHT"))) %>%
  rbind(data_yn_cht_wshape %>%
          group_by(day, type_chr) %>%
          summarise(mean_y = mean(log(rt)),
                    .groups = 'keep') %>%
          mutate(task = "Y/N (three stim categories w/ shape)",
                 sum_type = "group", subnum = 600,
                 type_chr = factor(type_chr, levels = c("HT", "RE", "CHT")))) %>%
  mutate(task = factor(task, levels = c("Y/N (two stim categories)", "Y/N (three stim categories w/ shape)")))

In [ ]:
p = ggplot()+
  geom_point(data = group_summary, aes(day, mean_y, color = type_chr))+

  geom_line(data = group_summary, aes(day, mean_y, color = type_chr))+
  
  geom_line(data = sub_summary, aes(day, mean_y, color = type_chr, linetype = as.factor(subnum)), alpha = .3)+
  
  facet_wrap(~task)+
  guides(linetype_chr = "none")+
  labs(x = "Day", y = "Mean Log RT", color="", shape = "", title = "Three stim types (w/ shape)")+
  theme(panel.grid.major.x = element_blank(),
        panel.grid.minor.x = element_blank(),
        panel.grid.minor.y = element_blank(),
        legend.position = "bottom",
        legend.box.margin=margin(t = -10),
        legend.spacing.y = unit(-0.25, "cm"),
        legend.box="vertical")+
  scale_x_continuous(breaks=seq(1,11,1))+
  scale_color_brewer(palette = "Dark2") +
  scale_linetype_manual(values = rep("solid", 6))+
  guides(linetype = "none")+
  annotate("rect", xmin = 2.75, xmax = 3.25, ymin = -.7, ymax = 1,
           alpha = .25,fill = "gray")+
  annotate("rect", xmin = 6.75, xmax = 7.25, ymin = -.7, ymax = 1,
           alpha = .25,fill = "gray")+
  annotate("rect", xmin = 10.75, xmax = 11.25, ymin = -.7, ymax = 1,
           alpha = .25,fill = "gray")

# ggsave(file=paste0(fig_out_path, 'yn_groupAndSub_meanLogRTOverDaysCHTwshape.jpg'), p, height = 5, width=12, units="in")
p

In [ ]:
p = sub_summary %>%
  filter(task =="Y/N (three stim categories w/ shape)") %>%
  ggplot(aes(day, mean_y, color = type_chr))+
  geom_point(position = position_dodge(width=.25))+
  geom_line(alpha =.5)+
  geom_errorbar(aes(ymin = mean_y - sem_y, ymax = mean_y + sem_y),position = position_dodge(width=.25), width = .2)+
  facet_wrap(~subnum)+
  labs(x = "Day", y = "Mean Log RT", color="", title = "Three stim types (w/ shape)")+
  theme(panel.grid.minor.x = element_blank(),
        panel.grid.major.x = element_blank(),
        legend.position = "bottom",
        legend.box.margin=margin(t = -10),
        legend.spacing.y = unit(-0.25, "cm"),
        legend.box="vertical")+
  scale_x_continuous(breaks=seq(1,11,1))+
  scale_color_brewer(palette = "Dark2")+
  annotate("rect", xmin = 2.75, xmax = 3.25, ymin = -1, ymax = 1,
           alpha = .25,fill = "gray")+
  annotate("rect", xmin = 6.75, xmax = 7.25, ymin = -1, ymax = 1,
           alpha = .25,fill = "gray")+
  annotate("rect", xmin = 10.75, xmax = 11.25, ymin = -1, ymax = 1,
           alpha = .25,fill = "gray")

# ggsave(file=paste0(fig_out_path, 'yn_sub_RTOverDaysCHTwshape.jpg'), p, height = 8, width=16, units="in")
p

In [ ]:
k = 3

my_fwd_diff = matrix(c(2/k, -1/k, -1/k,
                       1/k, 1/k, -2/k), ncol = k-1)

my_fwd_diff

In [ ]:
fn_draws = paste0(inputs_path, 'yn_rtOverDaysCHTwshape_draws.csv')

if(file.exists(fn_draws)){
  yn_rt_draws = read.csv(fn_draws)
} else{
  normMax = 1
  normMin = 0
  
  yn_rt_draws = tibble()
  
  for(cur_sub in unique(data_yn_clean$subnum)){
  
    cur_dat = data_yn_cht_wshape %>% 
      ungroup() %>%
      filter(subnum == cur_sub) %>%
      mutate(type_chr = factor(type_chr, levels = c("RE", "CHT", "HT")),
             norm_day = (normMax - normMin) / (max(day) - min(day)) * (day - max(day)) + (normMax),
             log_rt = log(rt))
    
    contrasts(cur_dat$type_chr) = my_fwd_diff
    
    cur_m <- brm(formula = log_rt ~  norm_day * type_chr,
                        data= cur_dat,
                        warmup = 500,
                        iter = 2500,
                        chains = 4,
                        init = "0",
                        cores = 4,
                        seed = 389253)
    
    cur_draws = as_draws_df(cur_m)
    cur_draws$subnum = cur_sub
    
    yn_rt_draws = rbind(yn_rt_draws, cur_draws)
  }
  names(yn_rt_draws) = gsub(":", ".", names(yn_rt_draws))
  write.csv(yn_rt_draws, fn_draws, row.names = F)

  rm(cur_dat, cur_m, cur_draws, cur_sub)
}


In [ ]:
names(as_tibble(yn_rt_draws))

In [ ]:
yn_par_summary = as_tibble(yn_rt_draws) %>%
  select(b_Intercept, b_norm_day, b_type_chr1, b_type_chr2, b_norm_day.type_chr1, b_norm_day.type_chr2, subnum) %>%
  gather(key, value, -subnum) %>%
  mutate(key=recode(key, 
                    "b_Intercept" = "Intercept", 
                    "b_norm_day" = "Day [0, 1]",
                    "b_type_chr1" = "Type (RE vs CHT)",
                    "b_type_chr2" = "Type (CHT vs HT)",
                    "b_norm_day.type_chr1" = "Day x Type (RE vs CHT)",
                    "b_norm_day.type_chr2" = "Day x Type (CHT vs HT)"),
         key = factor(key, levels = c("Intercept", "Day [0, 1]", "Type (RE vs CHT)", "Type (CHT vs HT)", "Day x Type (RE vs CHT)", "Day x Type (CHT vs HT)"))) %>%
  group_by(subnum, key) %>%
  summarise(.groups = 'keep',
            m = mean(value),
            l95 = HDInterval::hdi(value)[1],
            h95 = HDInterval::hdi(value)[2]) %>%
  mutate(subnum = as.factor(subnum),
         task = "Y/N") 

p = yn_par_summary %>%
  ggplot(aes(m, subnum)) +
  geom_vline(aes(xintercept = 0), color = "gray") + 
  geom_point(position = position_dodge(width=.75))+
  geom_errorbarh(aes(xmin = l95, xmax = h95, height=.2), position = position_dodge(width=.75))+
  facet_grid(.~key, scales = 'free_x') +
  labs(x = "Parameter Estimate", y = "Subject", color = "", title = "95% HDI for posterior slopes - log(RT) ~ Day x Type (3 levels; with shape)") +
  theme(legend.position = "bottom",
        panel.grid = element_blank(),
        legend.box.margin=margin(t = -10))+
  scale_x_symmetric()

# ggsave(file=paste0(fig_out_path, 'yn_RTCHTwshape_ParEsts.jpg'), p, height = 3, width=12, units="in")

p

## RT over value difference

In [ ]:
data_yn_cht_wshape = data_yn_cht_wshape %>%
  mutate(abs_val_diff_bin_str =  paste0(abs(val_diff_bin)*50-25,":",abs(val_diff_bin)*50+25),
         abs_val_diff_bin_str = ifelse(abs_val_diff_bin_str ==  "-25:25", "0:25", abs_val_diff_bin_str),
         abs_val_diff_bin_str = factor(abs_val_diff_bin_str, levels = c("0:25", "25:75", "75:125", "125:175", "175:225", "225:275")))

In [ ]:
sub_summary = data_yn_cht_wshape %>%
  mutate(time_bin = ifelse(day == 1, "Day 1", ifelse(day == 2, "Day 2", ifelse(day == 3, "Day 3", ifelse((day > 3) & (day < 8), "Week 2", "Week 3"))))) %>%
  group_by(subnum, time_bin, abs_val_diff_bin_str, type_chr) %>%
  summarise(mean_y = mean(log(rt)),
            sem_y = sd(log(rt))/sqrt(n()),
            .groups="keep") %>%
  mutate(type_chr = factor(type_chr, levels = c("HT", "CHT", "RE")))

group_summary = data_yn_cht_wshape %>%
  mutate(time_bin = ifelse(day == 1, "Day 1", ifelse(day == 2, "Day 2", ifelse(day == 3, "Day 3", ifelse((day > 3) & (day < 8), "Week 2", "Week 3"))))) %>%
  group_by(time_bin, abs_val_diff_bin_str, type_chr) %>%
  summarise(mean_y = mean(log(rt)),
            .groups="keep") %>%
  mutate(type_chr = factor(type_chr, levels = c("HT", "CHT", "RE")))

In [ ]:
p = ggplot() +
  geom_point(data = group_summary, aes(abs_val_diff_bin_str, mean_y, color = type_chr, alpha = time_bin, size = time_bin))+
  geom_line(data = group_summary, aes(abs_val_diff_bin_str, mean_y, color = type_chr, group = time_bin, alpha = time_bin)) +
  facet_wrap(~type_chr)+
  labs(x = "|Value Stim - Value Reference|", y = "Mean Log RT", title="Y/N Task (3 levels with shape)", alpha="", size = "")+
  # labs(x = "|Value Stim - Value Reference|", y = "Mean Log RT", alpha="", size = "")+
  theme(panel.grid.major.x  = element_blank(),
        panel.grid.minor.x  = element_blank(),
        panel.grid.minor.y  = element_blank(),
        legend.position = "bottom",
        axis.text.x = element_text(angle = 15, margin = margin(t = 5), size = 14),
        axis.text.y = element_text(size = 14),
        strip.text = element_text(size = 14),
        legend.box.margin=margin(t = -10),
        legend.spacing.y = unit(-0.25, "cm"),
        legend.box="vertical",
        legend.text = element_text(size=14),
        axis.title = element_text(size = 16))+
  scale_color_manual(values = c("#1B9E77", "#7570B3", "#D95F02" ))+
  scale_y_continuous(breaks = seq(-1,1,.5), limits = c(-1, 1))+
  guides(color = "none")+
  scale_alpha_manual(values = c(1, .6, .5, .3, .2))+
  scale_size_manual(values = c(1.5, 2 , 2.5, 3, 3.5))

# ggsave(file=paste0(fig_out_path, 'yn_group_MeanLogRTOverValDiffAcrossTimeCHTwshape.jpg'), p, height = 4, width = 12, units="in")

p

In [ ]:
p = ggplot() +
   geom_point(data = sub_summary, aes(abs_val_diff_bin_str, mean_y, color = type_chr, alpha = time_bin, size = time_bin))+
  geom_line(data = sub_summary, aes(abs_val_diff_bin_str, mean_y, color = type_chr, group = time_bin, alpha = time_bin)) +
  geom_errorbar(data = sub_summary, aes(x = abs_val_diff_bin_str, ymin = mean_y-sem_y, ymax = mean_y + sem_y, color = type_chr), width = 0, alpha = .6)+
  facet_grid(type_chr~subnum)+
  labs(x = "|Value Stim - Value Reference|", y = "Mean Log RT", title="Y/N Task (3 levels with shape)", alpha="", size = "")+
  theme(panel.grid.major.x  = element_blank(),
        panel.grid.minor.x  = element_blank(),
        panel.grid.minor.y  = element_blank(),
        legend.position = "bottom",
        axis.text.x = element_text(angle = 15, margin = margin(t = 5)),
        legend.box.margin=margin(t = -10),
        legend.spacing.y = unit(-0.25, "cm"),
        legend.box="vertical")+
  scale_color_manual(values = c("#1B9E77", "#7570B3", "#D95F02" ))+  
  scale_y_continuous(breaks = seq(-1,1,.5), limits = c(-1.3, 1.3))+
  guides(linetype_chr = "none", color = "none")+
  scale_alpha_manual(values = c(1, .6, .5, .3, .2))+
  scale_size_manual(values = c(1.5, 2 , 2.5, 3, 3.5))

# ggsave(file=paste0(fig_out_path, 'yn_subj_MeanLogRTValDiffAcrossTimeCHTwshape.jpg'), p, height = 7, width = 12, units="in")
p

In [ ]:
k = 3

my_fwd_diff = matrix(c(2/k, -1/k, -1/k,
                       1/k, 1/k, -2/k), ncol = k-1)

my_fwd_diff

In [ ]:
fn_draws = paste0(inputs_path, 'yn_rtValDiff_chtwshape_draws.csv')

if(file.exists(fn_draws)){
  yn_rt_draws = read.csv(fn_draws)
} else{
  
  normMax = 1
  normMin = 0
  
  yn_rt_draws = tibble()
  
  for(cur_sub in unique(data_yn_clean$subnum)){
  
    cur_dat = data_yn_cht_wshape %>% 
      ungroup() %>%
      filter(subnum == cur_sub) %>%
      mutate(type_chr = factor(type_chr, levels = c("RE", "CHT", "HT")),
             norm_day = (normMax - normMin) / (max(day) - min(day)) * (day - max(day)) + (normMax),
             log_rt = log(rt)) %>%
      group_by(day) %>% # Grouping by day to normalize valdiff matters. This makes sure every day ranges [-1,1]
      mutate(norm_absValDiff = (normMax - normMin) / (max(abs_val_diff) - min(abs_val_diff)) * (abs_val_diff - max(abs_val_diff)) + (normMax)) %>%
      ungroup() %>%
      select(log_rt, norm_absValDiff, norm_day, type_chr)
    
    contrasts(cur_dat$type_chr) = my_fwd_diff
    
    cur_m <- brm(formula = log_rt ~ norm_absValDiff * norm_day * type_chr,
                        data= cur_dat,
                        warmup = 500,
                        iter = 2500,
                        chains = 4,
                        init = "0",
                        cores = 4,
                        seed = 389253)
    
    cur_draws = as_draws_df(cur_m)
    cur_draws$subnum = cur_sub
    
    yn_rt_draws = rbind(yn_rt_draws, cur_draws)
  }
  names(yn_rt_draws) = gsub(":", ".", names(yn_rt_draws))
  write.csv(yn_rt_draws, fn_draws, row.names = F)

  rm(cur_dat, cur_m, cur_draws, cur_sub)
}


In [ ]:
names(as_tibble(yn_rt_draws))[grepl("b_", names(as_tibble(yn_rt_draws)))]

In [ ]:
yn_par_summary = as_tibble(yn_rt_draws) %>%
  select(b_Intercept, b_norm_absValDiff, b_norm_day, b_type_chr1, b_type_chr2, b_norm_absValDiff.norm_day, b_norm_absValDiff.type_chr1, b_norm_absValDiff.type_chr2, b_norm_day.type_chr1, b_norm_day.type_chr2, b_norm_absValDiff.norm_day.type_chr1, b_norm_absValDiff.norm_day.type_chr2, subnum) %>%
  gather(key, value, -subnum) %>%
  mutate(key=recode(key, 
                    "b_Intercept" = "Intercept", 
                    "b_norm_absValDiff"  = "|Value Diff| [0, 1]",
                    "b_norm_day" = "Day [0, 1]",
                    "b_type_chr1" = "Type (RE vs CHT)",
                    "b_type_chr2" = "Type (CHT vs HT)",
                    "b_norm_absValDiff.norm_day" = "|Value Diff| x Day",
                    "b_norm_absValDiff.type_chr1" = "|Value Diff| x Type (RE vs CHT)",
                    "b_norm_absValDiff.type_chr2" = "|Value Diff| x Type (CHT vs HT)",
                    "b_norm_day.type_chr1" = "Day x Type (RE vs CHT)",
                    "b_norm_day.type_chr2" = "Day x Type (CHT vs HT)",
                    "b_norm_absValDiff.norm_day.type_chr1" = "|Value Diff| x Day x Type (RE vs CHT)",
                    "b_norm_absValDiff.norm_day.type_chr2" = "|Value Diff| x Day x Type (CHT vs HT)",),
         key = factor(key, levels = c("Intercept", "|Value Diff| [0, 1]", "Day [0, 1]", "Type (RE vs CHT)", "Type (CHT vs HT)", "|Value Diff| x Day", "|Value Diff| x Type (RE vs CHT)", "|Value Diff| x Type (CHT vs HT)", "Day x Type (RE vs CHT)", "Day x Type (CHT vs HT)", "|Value Diff| x Day x Type (RE vs CHT)", "|Value Diff| x Day x Type (CHT vs HT)"))) %>%
  group_by(subnum, key) %>%
  summarise(.groups = 'keep',
            m = mean(value),
            l95 = HDInterval::hdi(value)[1],
            h95 = HDInterval::hdi(value)[2]) %>%
  mutate(subnum = as.factor(subnum),
         task = "Y/N") 

In [ ]:
p = yn_par_summary %>%
  ggplot(aes(m, subnum)) +
  geom_vline(aes(xintercept = 0), color = "gray") + 
  geom_point(position = position_dodge(width=.75))+
  geom_errorbarh(aes(xmin = l95, xmax = h95, height=.2), position = position_dodge(width=.75))+
  facet_grid(.~key, scales = 'free_x', labeller = labeller(key = label_wrap_gen(10))) +
  labs(x = "Parameter Estimate", y = "Subject", color = "", title = "95% HDI for posterior slopes - log(RT) ~ Value Diff x Day x Type (3 levels; with shape)") +
  theme(legend.position = "bottom",
        panel.grid = element_blank(),
        legend.box.margin=margin(t = -10))+
  scale_x_symmetric()

# ggsave(file=paste0(fig_out_path, 'yn_rtvalDiff_chtwshape_ParEsts.jpg'), p, height = 3, width=12, units="in")

p

# Definition without shape

In [ ]:
get_sub_cht_stims_for_one_ht_noshape = function(cur_sub, ht_stim){
  
  # For each of the HT stims define CHT stims as:
  # same shape, same orientation. filling level +/- 1 (2 stims)
  # same shape, filling, orientation level +/1 (2 stims)
  # Exclude if any CHT is also an HT stim
  
  sub_ht_stim = data_yn_clean %>%
    filter(subnum == cur_sub & type == 1 & stimNum == ht_stim) %>%
    select(type, stimNum, shape, filling, orientation) %>%
    distinct()
  
  shape_level = sub_ht_stim$shape[1]
  orientation_level = sub_ht_stim$orientation[1]
  filling_level = sub_ht_stim$filling[1]
  
  # List all levels of all attributes
  orientation_levels = sort(unique(data_yn_clean$orientation))
  filling_levels = sort(unique(data_yn_clean$filling))
  
  out = tibble()

  # Get close orientation levels
  orientation_level_index = which(orientation_levels == orientation_level)  
  
  if(orientation_level_index == 1){  # if o = 0, get o = 15, o = 150
    close_orientation_levels = c(orientation_levels[2], orientation_levels[length(orientation_levels)])
  } else if (orientation_level_index == length(orientation_levels)){ #if o = 150, get o = 135, o = 0
    close_orientation_levels = c(orientation_levels[1], orientation_levels[length(orientation_levels) - 1])  
  } else{
    close_orientation_levels = orientation_levels[c(orientation_level_index-1, orientation_level_index+1)]
  }
  
  out = rbind(out, tibble(shape = shape_level, filling = filling_level, orientation = close_orientation_levels))
  
  
  # Get close filling levels
  filling_level_index = which(filling_levels == filling_level)
  
  if(filling_level_index == 1){
    close_filling_levels = filling_levels[2]
  } else if(filling_level_index == length(filling_levels)){
    close_filling_levels = filling_levels[filling_level_index - 1]
  } else {
    close_filling_levels = filling_levels[c(filling_level_index - 1, filling_level_index + 1)]
  }
  
  out = rbind(out, tibble(shape = shape_level, filling = close_filling_levels, orientation = orientation_level))
  
  # Get stimNums for the CHT 
  out = data_yn_clean %>%
    filter(subnum == cur_sub) %>%
    select(stimNum, type, shape, filling, orientation) %>%
    right_join(out, by = c("shape", "filling", "orientation")) %>%
    arrange(stimNum) %>%
    
    # Check if any of the CHT are also HT
    filter(type != 1) %>%
    distinct()
  
  return(out)
}

Wrapper to get all close to HT stims for a given subject

In [ ]:
get_sub_cht_stims_for_all_ht_noshape = function(cur_sub){
  
  sub_ht_stims = get_sub_ht_stims(cur_sub)
  
  out = tibble()
  
  for(cur_ht_stim in sub_ht_stims$stimNum){
    out = rbind(out, get_sub_cht_stims_for_one_ht_noshape(cur_sub, cur_ht_stim))
  }
  
  return(out)
  
}

In [ ]:
get_sub_cht_stims_for_all_ht_noshape(601)

## Assign CHT labels to dataset

In [ ]:

subnums = unique(data_yn_clean$subnum)
out = tibble()
for(cur_sub in subnums){
  sub_cht_stimnums = get_sub_cht_stims_for_all_ht_noshape(cur_sub)
  
  sub_cht_stimnums = sub_cht_stimnums$stimNum
  
  cur_out = data_yn_clean %>%
    filter(subnum == cur_sub) %>%
    mutate(type_chr = ifelse(stimNum %in% sub_cht_stimnums, "CHT", type_chr))
  
  out = rbind(out, cur_out)
}

data_yn_cht_noshape = out
rm(out, subnums, sub_cht_stimnums, cur_out)


In [ ]:
with(data_yn_cht_noshape, table(subnum, type_chr))

## Accuracy for CHT

In [ ]:
sub_summary = data_yn_clean %>%
  group_by(day, subnum, type_chr) %>%
  summarise(mean_accuracy = mean(correct),
            sem_accuracy = sd(correct)/sqrt(n()),
            .groups = 'keep') %>%
  mutate(task = "Y/N (two stim categories)",
         sum_type = "sub",
         type_chr = factor(type_chr, levels = c("HT", "RE", "CHT"))) %>%
  rbind(data_yn_cht_noshape %>%
  group_by(day, subnum, type_chr) %>%
  summarise(mean_accuracy = mean(correct),
            sem_accuracy = sd(correct)/sqrt(n()),
            .groups = 'keep') %>%
  mutate(task = "Y/N (three stim categories no shape)",
         sum_type = "sub",
         type_chr = factor(type_chr, levels = c("HT", "RE", "CHT")))) %>%
  mutate(task = factor(task, levels = c("Y/N (two stim categories)", "Y/N (three stim categories no shape)")))

group_summary = data_yn_clean %>%
  group_by(day, type_chr) %>%
  summarise(mean_accuracy = mean(correct),
            .groups = 'keep') %>%
  mutate(task = "Y/N (two stim categories)",
         sum_type = "group", subnum = 600,
         type_chr = factor(type_chr, levels = c("HT", "RE", "CHT"))) %>%
  rbind(data_yn_cht_noshape %>%
          group_by(day, type_chr) %>%
          summarise(mean_accuracy = mean(correct),
                    .groups = 'keep') %>%
          mutate(task = "Y/N (three stim categories no shape)",
                 sum_type = "group", subnum = 600,
                 type_chr = factor(type_chr, levels = c("HT", "RE", "CHT")))) %>%
  mutate(task = factor(task, levels = c("Y/N (two stim categories)", "Y/N (three stim categories no shape)")))

In [ ]:
p = ggplot()+
  geom_point(data = group_summary, aes(day, mean_accuracy, color = type_chr))+

  geom_line(data = group_summary, aes(day, mean_accuracy, color = type_chr))+
  
  geom_line(data = sub_summary, aes(day, mean_accuracy, color = type_chr, linetype = as.factor(subnum)), alpha = .3)+

  facet_wrap(~task)+
  labs(x = "Day", y = "Accuracy", color="", title = "Three stim types (no shape)")+
  theme(panel.grid.minor.x = element_blank(),
        panel.grid.major.x = element_blank(),
        legend.position = "bottom",
        legend.box.margin=margin(t = -10),
        legend.spacing.y = unit(-0.25, "cm"),
        legend.box="vertical")+
  scale_x_continuous(breaks=seq(1,11,1))+
  scale_y_continuous(breaks=seq(.7,1,.1), limits = c(.7, 1))+
  scale_color_brewer(palette = "Dark2")+
  scale_linetype_manual(values = rep("solid", 6))+
  guides(linetype = "none")+
  annotate("rect", xmin = 2.75, xmax = 3.25, ymin = .7, ymax = 1,
           alpha = .25,fill = "gray")+
  annotate("rect", xmin = 6.75, xmax = 7.25, ymin = .7, ymax = 1,
           alpha = .25,fill = "gray")+
  annotate("rect", xmin = 10.75, xmax = 11.25, ymin = .7, ymax = 1,
           alpha = .25,fill = "gray")

# ggsave(file=paste0(fig_out_path, 'yn_groupAndSub_accuracyOverDaysCHTnoshape.jpg'), p, height = 5, width=12, units="in")
p

In [ ]:
p = sub_summary %>%
  filter(task =="Y/N (three stim categories no shape)") %>%
  ggplot(aes(day, mean_accuracy, color = type_chr))+
  geom_point(position = position_dodge(width=.25))+
  geom_line(alpha =.5)+
  geom_errorbar(aes(ymin = mean_accuracy - sem_accuracy, ymax = mean_accuracy + sem_accuracy),position = position_dodge(width=.25), width = .2)+
  facet_wrap(~subnum)+
  labs(x = "Day", y = "Accuracy", color="", title = "Three stim types (no shape)")+
  theme(panel.grid.minor.x = element_blank(),
        panel.grid.major.x = element_blank(),
        legend.position = "bottom",
        legend.box.margin=margin(t = -10),
        legend.spacing.y = unit(-0.25, "cm"),
        legend.box="vertical")+
  scale_x_continuous(breaks=seq(1,11,1))+
  scale_y_continuous(breaks=seq(.7,1,.1), limits = c(.6, 1))+
  scale_color_brewer(palette = "Dark2")+
 
  annotate("rect", xmin = 2.75, xmax = 3.25, ymin = .6, ymax = 1,
           alpha = .25,fill = "gray")+
  annotate("rect", xmin = 6.75, xmax = 7.25, ymin = .6, ymax = 1,
           alpha = .25,fill = "gray")+
  annotate("rect", xmin = 10.75, xmax = 11.25, ymin = .6, ymax = 1,
           alpha = .25,fill = "gray")

# ggsave(file=paste0(fig_out_path, 'yn_sub_accuracyOverDaysCHTnoshape.jpg'), p, height = 8, width=16, units="in")
p

## Model for accuracy difference

In [ ]:
fn_draws = paste0(inputs_path, 'yn_correct_cht_noshape_fwd_diff_draws.csv')

if(file.exists(fn_draws)){
  yn_acc_draws = read.csv(fn_draws)
} else{
  subnums = unique(data_yn_cht_noshape$subnum)

  normMax = 1
  normMin = 0

  yn_acc_draws = tibble()

  for(cur_sub in subnums){
    
    cur_dat = data_yn_cht_noshape %>% 
      ungroup() %>%
      filter(subnum == cur_sub) %>%
      mutate(type_chr = factor(type_chr, levels = c("RE", "CHT", "HT")),
             norm_day = (normMax - normMin) / (max(day) - min(day)) * (day - max(day)) + (normMax))
    
    contrasts(cur_dat$type_chr) = my_fwd_diff
    
    cur_m <- brm(formula = correct ~  norm_day * type_chr,
                        data= cur_dat,
                        family = bernoulli(link = "logit"),
                        warmup = 500,
                        iter = 2500,
                        chains = 4,
                        init = "0",
                        cores = 4,
                        seed = 389253)

    cur_draws = as_draws_df(cur_m)
    cur_draws$subnum = cur_sub

    yn_acc_draws = rbind(yn_acc_draws, cur_draws)

  }
  names(yn_acc_draws) = gsub(":", ".", names(yn_acc_draws))
  write.csv(yn_acc_draws, fn_draws, row.names = F)

  rm(cur_dat, cur_m, cur_draws, cur_sub)
}

In [ ]:
yn_par_summary = as_tibble(yn_acc_draws) %>%
  select(b_Intercept, b_norm_day, b_type_chr1, b_type_chr2, b_norm_day.type_chr1, b_norm_day.type_chr2, subnum) %>%
  gather(key, value, -subnum) %>%
  mutate(key=recode(key, 
                    "b_Intercept" = "Intercept", 
                    "b_norm_day" = "Day [0, 1]",
                    "b_type_chr1" = "Type (RE vs CHT)",
                    "b_type_chr2" = "Type (CHT vs HT)",
                    "b_norm_day.type_chr1" = "Day x Type (RE vs CHT)",
                    "b_norm_day.type_chr2" = "Day x Type (CHT vs HT)"),
         key = factor(key, levels = c("Intercept", "Day [0, 1]", "Type (RE vs CHT)", "Type (CHT vs HT)", "Day x Type (RE vs CHT)", "Day x Type (CHT vs HT)"))) %>%
  group_by(subnum, key) %>%
  summarise(.groups = 'keep',
            m = mean(value),
            l95 = HDInterval::hdi(value)[1],
            h95 = HDInterval::hdi(value)[2]) %>%
  mutate(subnum = as.factor(subnum),
         task = "Y/N") 

p = yn_par_summary %>%
  ggplot(aes(m, subnum)) +
  geom_vline(aes(xintercept = 0), color = "gray") + 
  geom_point(position = position_dodge(width=.75))+
  geom_errorbarh(aes(xmin = l95, xmax = h95, height=.2), position = position_dodge(width=.75))+
  facet_grid(.~key, scales = 'free_x') +
  labs(x = "Parameter Estimate", y = "Subject", color = "", title = "95% HDI for posterior slopes - Correct ~ Day x Type (3 levels; no shape)") +
  theme(legend.position = "bottom",
        panel.grid = element_blank(),
        legend.box.margin=margin(t = -10))+
  scale_x_symmetric()

# ggsave(file=paste0(fig_out_path, 'yn_accCHTnoshape_ParEsts.jpg'), p, height = 3, width=12, units="in")

p

## Psychometric plots for CHT

In [ ]:
sub_summary = data_yn_cht_noshape %>%
  mutate(time_bin = ifelse(day == 1, "Day 1", ifelse(day == 2, "Day 2", ifelse(day == 3, "Day 3", ifelse((day > 3) & (day < 8), "Week 2", "Week 3"))))) %>%
  group_by(subnum, time_bin, val_diff_bin_str, type_chr) %>%
  summarise(mean_y = mean(yesChosen),
            sem_y = sd(yesChosen)/sqrt(n()),
            .groups="keep") %>%
  mutate(type_chr = factor(type_chr, levels = c("HT", "CHT", "RE")))

group_summary = data_yn_cht_noshape %>%
  mutate(time_bin = ifelse(day == 1, "Day 1", ifelse(day == 2, "Day 2", ifelse(day == 3, "Day 3", ifelse((day > 3) & (day < 8), "Week 2", "Week 3"))))) %>%
  group_by(time_bin, val_diff_bin_str, type_chr) %>%
  summarise(mean_y = mean(yesChosen),
            .groups="keep")%>%
    mutate(type_chr = factor(type_chr, levels = c("HT", "CHT", "RE")))

In [ ]:
p = ggplot() +
  geom_hline(data = group_summary, aes(yintercept =.5), color="gray")+
  geom_vline(data = group_summary, aes(xintercept = 5), color="gray")+
  geom_point(data = group_summary, aes(val_diff_bin_str, mean_y, color = type_chr, alpha = time_bin, size = time_bin))+
  geom_line(data = group_summary, aes(val_diff_bin_str, mean_y, color = type_chr, group = time_bin, alpha = time_bin)) +
  facet_wrap(~type_chr)+
  # labs(x = "Value Stim - Value Reference", y = "p(Yes)", title="Y/N Task", alpha="", size = "")+
    labs(x = "Value Stim - Value Reference", y = "p(Yes)", alpha="", size = "", title = "Y/N Task (3 levels no shape)")+
  theme(panel.grid = element_blank(),
        legend.position = "bottom",
        axis.text.x = element_text(angle = 15, margin = margin(t = 5), size = 14),
        axis.text.y = element_text(size = 14),
        strip.text = element_text(size = 14),
        legend.box.margin=margin(t = -10),
        legend.spacing.y = unit(-0.25, "cm"),
        legend.box="vertical",
        legend.text = element_text(size=14),
        axis.title = element_text(size = 16))+
  # scale_color_brewer(palette = "Dark2")+
  scale_color_manual(values = c("#1B9E77", "#7570B3", "#D95F02" ))+
  scale_x_discrete(labels=c("-225:-175", "", "-125:-75", "", "-25:25", "", "75:125", "", "175:225"))+
  scale_y_continuous(breaks = seq(0,1,.5))+
  guides(linetype = "none", color = "none")+
  scale_alpha_manual(values = c(1, .6, .5, .3, .2))+
  scale_size_manual(values = c(1.5, 2 , 2.5, 3, 3.5))

# ggsave(file=paste0(fig_out_path, 'yn_group_pYesOverValDiffAcrossTimeCHTnoShape.jpg'), p, height = 4, width = 12, units="in")

p

In [ ]:
p = ggplot() +
  geom_hline(data = sub_summary, aes(yintercept =.5), color="gray")+
  geom_vline(data = sub_summary, aes(xintercept = 5), color="gray")+
  geom_point(data = sub_summary, aes(val_diff_bin_str, mean_y, color = type_chr, alpha = time_bin, size = time_bin))+
  geom_line(data = sub_summary, aes(val_diff_bin_str, mean_y, color = type_chr, group = time_bin, alpha = time_bin)) +
  geom_errorbar(data = sub_summary, aes(x = val_diff_bin_str, ymin = mean_y-sem_y, ymax = mean_y + sem_y, color = type_chr), width = 0, alpha = .6)+
  facet_grid(type_chr~subnum)+
  labs(x = "Value Stim - Value Reference", y = "p(Yes)", title="Y/N Task (3 levels no shape)", alpha="", size = "")+
  theme(panel.grid = element_blank(),
        legend.position = "bottom",
        axis.text.x = element_text(angle = 15, margin = margin(t = 5)),
        legend.box.margin=margin(t = -10),
        legend.spacing.y = unit(-0.25, "cm"),
        legend.box="vertical")+
  scale_color_manual(values = c("#1B9E77", "#7570B3", "#D95F02" ))+
  scale_x_discrete(labels=c("-225:-175", "", "-125:-75", "", "-25:25", "", "75:125", "", "175:225"))+
  scale_y_continuous(breaks = seq(0,1,.5))+
  guides(linetype = "none", color = "none")+
  scale_alpha_manual(values = c(1, .6, .5, .3, .2))+
  scale_size_manual(values = c(1.5, 2 , 2.5, 3, 3.5))

# ggsave(file=paste0(fig_out_path, 'yn_subj_pYesOverValDiffAcrossTimeCHTnoShape.jpg'), p, height = 7, width = 12, units="in")
p

## Logits for CHT

In [ ]:
k = 3

my_fwd_diff = matrix(c(2/k, -1/k, -1/k,
                       1/k, 1/k, -2/k), ncol = k-1)

my_fwd_diff

In [ ]:
fn_draws = paste0(inputs_path, 'yn_logit_norm_cht_noshape_draws.csv')

if(file.exists(fn_draws)){
  yn_logit_draws = read.csv(fn_draws)
} else{
  normMax = 1
  normMinDay = 0
  normMinVal = -1
  
  yn_logit_draws = tibble()
  
  for(cur_sub in unique(data_yn_clean$subnum)){
  
    cur_dat = data_yn_cht_noshape %>% 
      ungroup() %>%
      filter(subnum == cur_sub) %>%
      mutate(type_chr = factor(type_chr, levels = c("RE", "CHT", "HT")),
             norm_day = (normMax - normMinDay) / (max(day) - min(day)) * (day - max(day)) + (normMax)) %>%
      group_by(day) %>% # Grouping by day to normalize valdiff matters. This makes sure every day ranges [-1,1]
      mutate(norm_valDiff = (normMax - normMinVal) / (max(val_diff) - min(val_diff)) * (val_diff - max(val_diff)) + (normMax)) %>%
      select(yesChosen, norm_valDiff, norm_day, type_chr)
    
    contrasts(cur_dat$type_chr) = my_fwd_diff
    
    cur_m <- brm(formula = yesChosen ~ norm_valDiff * norm_day * type_chr,
                        data= cur_dat,
                        family = bernoulli(link = "logit"),
                        warmup = 500,
                        iter = 2500,
                        chains = 4,
                        init = "0",
                        cores = 4,
                        seed = 389253)
    
    cur_draws = as_draws_df(cur_m)
    cur_draws$subnum = cur_sub
    
    yn_logit_draws = rbind(yn_logit_draws, cur_draws)
  }
  names(yn_logit_draws) = gsub(":", ".", names(yn_logit_draws))
  write.csv(yn_logit_draws, fn_draws, row.names = F)

  rm(cur_dat, cur_m, cur_draws, cur_sub)
}

In [ ]:
names(as_tibble(yn_logit_draws))

In [ ]:
yn_par_summary = as_tibble(yn_logit_draws) %>%
  select(b_Intercept, b_norm_day, b_norm_valDiff, b_type_chr1, b_type_chr2, b_norm_valDiff.norm_day, b_norm_valDiff.type_chr1, b_norm_valDiff.type_chr2, b_norm_day.type_chr1, b_norm_day.type_chr2, b_norm_valDiff.norm_day.type_chr1, b_norm_valDiff.norm_day.type_chr2, subnum) %>%
  gather(key, value, -subnum) %>%
  mutate(key=recode(key, 
                    "b_Intercept" = "Intercept", 
                    "b_norm_day" = "Day [0, 1]",
                    "b_norm_valDiff" = "Value Diff [-1, 1]",
                    "b_type_chr1" = "Type (RE vs CHT)",
                    "b_type_chr2" = "Type (CHT vs HT)",
                    "b_norm_valDiff.norm_day" = "Value Diff x Day",
                    "b_norm_valDiff.type_chr1" = "Value Diff x Type (RE vs CHT)",
                    "b_norm_valDiff.type_chr2" = "Value Diff x Type (CHT vs HT)",
                    "b_norm_day.type_chr1" = "Day x Type (RE vs CHT)",
                    "b_norm_day.type_chr2" = "Day x Type (CHT vs HT)",
                    "b_norm_valDiff.norm_day.type_chr1" = "Value Diff x Day x Type (RE vs CHT)",
                    "b_norm_valDiff.norm_day.type_chr2" = "Value Diff x Day x Type (CHT vs HT)"),
         key = factor(key, levels = c("Intercept", "Day [0, 1]", "Value Diff [-1, 1]", "Type (RE vs CHT)", "Type (CHT vs HT)", "Value Diff x Day", "Value Diff x Type (RE vs CHT)", "Value Diff x Type (CHT vs HT)", "Day x Type (RE vs CHT)", "Day x Type (CHT vs HT)", "Value Diff x Day x Type (RE vs CHT)", "Value Diff x Day x Type (CHT vs HT)"))) %>%
  group_by(subnum, key) %>%
  summarise(.groups = 'keep',
            m = mean(value),
            l95 = HDInterval::hdi(value)[1],
            h95 = HDInterval::hdi(value)[2]) %>%
  mutate(subnum = as.factor(subnum),
         task = "Y/N") 

p = yn_par_summary %>%
  ggplot(aes(m, subnum)) +
  geom_vline(aes(xintercept = 0), color = "gray") + 
  geom_point(position = position_dodge(width=.75))+
  geom_errorbarh(aes(xmin = l95, xmax = h95, height=.2), position = position_dodge(width=.75))+
  facet_grid(.~key, scales = 'free_x', labeller = labeller(key = label_wrap_gen(10))) +
  labs(x = "Parameter Estimate", y = "Subject", color = "", title = "95% HDI for posterior slopes - Yes ~ Value Diff x Day x Type (3 levels; no shape)") +
  theme(legend.position = "bottom",
        panel.grid = element_blank(),
        legend.box.margin=margin(t = -10))+
  scale_x_symmetric()

# ggsave(file=paste0(fig_out_path, 'yn_logitCHTnoshape_ParEsts.jpg'), p, height = 3, width=12, units="in")

p

## RT over days

In [ ]:
sub_summary = data_yn_clean %>%
  group_by(day, subnum, type_chr) %>%
  summarise(mean_y = mean(log(rt)),
            sem_y = sd(log(rt))/sqrt(n()),
            .groups = 'keep') %>%
  mutate(task = "Y/N (two stim categories)",
         sum_type = "sub",
         type_chr = factor(type_chr, levels = c("HT", "RE", "CHT"))) %>%
  rbind(data_yn_cht_noshape %>%
  group_by(day, subnum, type_chr) %>%
  summarise(mean_y = mean(log(rt)),
            sem_y = sd(log(rt))/sqrt(n()),
            .groups = 'keep') %>%
  mutate(task = "Y/N (three stim categories no shape)",
         sum_type = "sub",
         type_chr = factor(type_chr, levels = c("HT", "RE", "CHT")))) %>%
  mutate(task = factor(task, levels = c("Y/N (two stim categories)", "Y/N (three stim categories no shape)")))

group_summary = data_yn_clean %>%
  group_by(day, type_chr) %>%
  summarise(mean_y = mean(log(rt)),
            .groups = 'keep') %>%
  mutate(task = "Y/N (two stim categories)",
         sum_type = "group", subnum = 600,
         type_chr = factor(type_chr, levels = c("HT", "RE", "CHT"))) %>%
  rbind(data_yn_cht_noshape %>%
          group_by(day, type_chr) %>%
          summarise(mean_y = mean(log(rt)),
                    .groups = 'keep') %>%
          mutate(task = "Y/N (three stim categories no shape)",
                 sum_type = "group", subnum = 600,
                 type_chr = factor(type_chr, levels = c("HT", "RE", "CHT")))) %>%
  mutate(task = factor(task, levels = c("Y/N (two stim categories)", "Y/N (three stim categories no shape)")))

In [ ]:
p = ggplot()+
  geom_point(data = group_summary, aes(day, mean_y, color = type_chr))+

  geom_line(data = group_summary, aes(day, mean_y, color = type_chr))+
  
  geom_line(data = sub_summary, aes(day, mean_y, color = type_chr, linetype = as.factor(subnum)), alpha = .3)+
  
  facet_wrap(~task)+
  guides(linetype_chr = "none")+
  labs(x = "Day", y = "Mean Log RT", color="", shape = "", title = "Three stim types (no shape)")+
  theme(panel.grid.major.x = element_blank(),
        panel.grid.minor.x = element_blank(),
        panel.grid.minor.y = element_blank(),
        legend.position = "bottom",
        legend.box.margin=margin(t = -10),
        legend.spacing.y = unit(-0.25, "cm"),
        legend.box="vertical")+
  scale_x_continuous(breaks=seq(1,11,1))+
  scale_color_brewer(palette = "Dark2") +
  scale_linetype_manual(values = rep("solid", 6))+
  guides(linetype = "none")+
  annotate("rect", xmin = 2.75, xmax = 3.25, ymin = -.7, ymax = 1,
           alpha = .25,fill = "gray")+
  annotate("rect", xmin = 6.75, xmax = 7.25, ymin = -.7, ymax = 1,
           alpha = .25,fill = "gray")+
  annotate("rect", xmin = 10.75, xmax = 11.25, ymin = -.7, ymax = 1,
           alpha = .25,fill = "gray")

# ggsave(file=paste0(fig_out_path, 'yn_groupAndSub_meanLogRTOverDaysCHTnoshape.jpg'), p, height = 5, width=12, units="in")
p

In [ ]:
p = sub_summary %>%
  filter(task =="Y/N (three stim categories no shape)") %>%
  ggplot(aes(day, mean_y, color = type_chr))+
  geom_point(position = position_dodge(width=.25))+
  geom_line(alpha =.5)+
  geom_errorbar(aes(ymin = mean_y - sem_y, ymax = mean_y + sem_y),position = position_dodge(width=.25), width = .2)+
  facet_wrap(~subnum)+
  labs(x = "Day", y = "Mean Log RT", color="", title = "Three stim types (no shape)")+
  theme(panel.grid.minor.x = element_blank(),
        panel.grid.major.x = element_blank(),
        legend.position = "bottom",
        legend.box.margin=margin(t = -10),
        legend.spacing.y = unit(-0.25, "cm"),
        legend.box="vertical")+
  scale_x_continuous(breaks=seq(1,11,1))+
  scale_color_brewer(palette = "Dark2")+
  annotate("rect", xmin = 2.75, xmax = 3.25, ymin = -1.5, ymax = 1,
           alpha = .25,fill = "gray")+
  annotate("rect", xmin = 6.75, xmax = 7.25, ymin = -1.5, ymax = 1,
           alpha = .25,fill = "gray")+
  annotate("rect", xmin = 10.75, xmax = 11.25, ymin = -1.5, ymax = 1,
           alpha = .25,fill = "gray")

# ggsave(file=paste0(fig_out_path, 'yn_sub_RTOverDaysCHTnoshape.jpg'), p, height = 8, width=16, units="in")
p

In [ ]:
k = 3

my_fwd_diff = matrix(c(2/k, -1/k, -1/k,
                       1/k, 1/k, -2/k), ncol = k-1)

my_fwd_diff

In [ ]:
fn_draws = paste0(inputs_path, 'yn_rtOverDaysCHTnoshape_draws.csv')

if(file.exists(fn_draws)){
  yn_rt_draws = read.csv(fn_draws)
} else{
  normMax = 1
  normMin = 0
  
  yn_rt_draws = tibble()
  
  for(cur_sub in unique(data_yn_clean$subnum)){
  
    cur_dat = data_yn_cht_noshape %>% 
      ungroup() %>%
      filter(subnum == cur_sub) %>%
      mutate(type_chr = factor(type_chr, levels = c("RE", "CHT", "HT")),
             norm_day = (normMax - normMin) / (max(day) - min(day)) * (day - max(day)) + (normMax),
             log_rt = log(rt))
    
    contrasts(cur_dat$type_chr) = my_fwd_diff
    
    cur_m <- brm(formula = log_rt ~  norm_day * type_chr,
                        data= cur_dat,
                        warmup = 500,
                        iter = 2500,
                        chains = 4,
                        init = "0",
                        cores = 4,
                        seed = 389253)
    
    cur_draws = as_draws_df(cur_m)
    cur_draws$subnum = cur_sub
    
    yn_rt_draws = rbind(yn_rt_draws, cur_draws)
  }
  names(yn_rt_draws) = gsub(":", ".", names(yn_rt_draws))
  write.csv(yn_rt_draws, fn_draws, row.names = F)

  rm(cur_dat, cur_m, cur_draws, cur_sub)
}


In [ ]:
names(as_tibble(yn_rt_draws))

In [ ]:
yn_par_summary = as_tibble(yn_rt_draws) %>%
  select(b_Intercept, b_norm_day, b_type_chr1, b_type_chr2, b_norm_day.type_chr1, b_norm_day.type_chr2, subnum) %>%
  gather(key, value, -subnum) %>%
  mutate(key=recode(key, 
                    "b_Intercept" = "Intercept", 
                    "b_norm_day" = "Day [0, 1]",
                    "b_type_chr1" = "Type (RE vs CHT)",
                    "b_type_chr2" = "Type (CHT vs HT)",
                    "b_norm_day.type_chr1" = "Day x Type (RE vs CHT)",
                    "b_norm_day.type_chr2" = "Day x Type (CHT vs HT)"),
         key = factor(key, levels = c("Intercept", "Day [0, 1]", "Type (RE vs CHT)", "Type (CHT vs HT)", "Day x Type (RE vs CHT)", "Day x Type (CHT vs HT)"))) %>%
  group_by(subnum, key) %>%
  summarise(.groups = 'keep',
            m = mean(value),
            l95 = HDInterval::hdi(value)[1],
            h95 = HDInterval::hdi(value)[2]) %>%
  mutate(subnum = as.factor(subnum),
         task = "Y/N") 

p = yn_par_summary %>%
  ggplot(aes(m, subnum)) +
  geom_vline(aes(xintercept = 0), color = "gray") + 
  geom_point(position = position_dodge(width=.75))+
  geom_errorbarh(aes(xmin = l95, xmax = h95, height=.2), position = position_dodge(width=.75))+
  facet_grid(.~key, scales = 'free_x') +
  labs(x = "Parameter Estimate", y = "Subject", color = "", title = "95% HDI for posterior slopes - log(RT) ~ Day x Type (3 levels; no shape)") +
  theme(legend.position = "bottom",
        panel.grid = element_blank(),
        legend.box.margin=margin(t = -10))+
  scale_x_symmetric()

# ggsave(file=paste0(fig_out_path, 'yn_RTCHTnoshape_ParEsts.jpg'), p, height = 3, width=12, units="in")

p

## RT over value difference

In [ ]:
data_yn_cht_noshape = data_yn_cht_noshape %>%
  mutate(abs_val_diff_bin_str =  paste0(abs(val_diff_bin)*50-25,":",abs(val_diff_bin)*50+25),
         abs_val_diff_bin_str = ifelse(abs_val_diff_bin_str ==  "-25:25", "0:25", abs_val_diff_bin_str),
         abs_val_diff_bin_str = factor(abs_val_diff_bin_str, levels = c("0:25", "25:75", "75:125", "125:175", "175:225", "225:275")))

In [ ]:
sub_summary = data_yn_cht_noshape %>%
  mutate(time_bin = ifelse(day == 1, "Day 1", ifelse(day == 2, "Day 2", ifelse(day == 3, "Day 3", ifelse((day > 3) & (day < 8), "Week 2", "Week 3"))))) %>%
  group_by(subnum, time_bin, abs_val_diff_bin_str, type_chr) %>%
  summarise(mean_y = mean(log(rt)),
            sem_y = sd(log(rt))/sqrt(n()),
            .groups="keep") %>%
  mutate(type_chr = factor(type_chr, levels = c("HT", "CHT", "RE")))

group_summary = data_yn_cht_noshape %>%
  mutate(time_bin = ifelse(day == 1, "Day 1", ifelse(day == 2, "Day 2", ifelse(day == 3, "Day 3", ifelse((day > 3) & (day < 8), "Week 2", "Week 3"))))) %>%
  group_by(time_bin, abs_val_diff_bin_str, type_chr) %>%
  summarise(mean_y = mean(log(rt)),
            .groups="keep") %>%
  mutate(type_chr = factor(type_chr, levels = c("HT", "CHT", "RE")))

In [ ]:
p = ggplot() +
  geom_point(data = group_summary, aes(abs_val_diff_bin_str, mean_y, color = type_chr, alpha = time_bin, size = time_bin))+
  geom_line(data = group_summary, aes(abs_val_diff_bin_str, mean_y, color = type_chr, group = time_bin, alpha = time_bin)) +
  facet_wrap(~type_chr)+
  labs(x = "|Value Stim - Value Reference|", y = "Mean Log RT", title="Y/N Task (3 levels no shape)", alpha="", size = "")+
  # labs(x = "|Value Stim - Value Reference|", y = "Mean Log RT", alpha="", size = "")+
  theme(panel.grid.major.x  = element_blank(),
        panel.grid.minor.x  = element_blank(),
        panel.grid.minor.y  = element_blank(),
        legend.position = "bottom",
        axis.text.x = element_text(angle = 15, margin = margin(t = 5), size = 14),
        axis.text.y = element_text(size = 14),
        strip.text = element_text(size = 14),
        legend.box.margin=margin(t = -10),
        legend.spacing.y = unit(-0.25, "cm"),
        legend.box="vertical",
        legend.text = element_text(size=14),
        axis.title = element_text(size = 16))+
  scale_color_manual(values = c("#1B9E77", "#7570B3", "#D95F02" ))+
  scale_y_continuous(breaks = seq(-1,1,.5), limits = c(-1, 1))+
  guides(color = "none")+
  scale_alpha_manual(values = c(1, .6, .5, .3, .2))+
  scale_size_manual(values = c(1.5, 2 , 2.5, 3, 3.5))

# ggsave(file=paste0(fig_out_path, 'yn_group_MeanLogRTOverValDiffAcrossTimeCHTnoshape.jpg'), p, height = 4, width = 12, units="in")

p

In [ ]:
p = ggplot() +
   geom_point(data = sub_summary, aes(abs_val_diff_bin_str, mean_y, color = type_chr, alpha = time_bin, size = time_bin))+
  geom_line(data = sub_summary, aes(abs_val_diff_bin_str, mean_y, color = type_chr, group = time_bin, alpha = time_bin)) +
  geom_errorbar(data = sub_summary, aes(x = abs_val_diff_bin_str, ymin = mean_y-sem_y, ymax = mean_y + sem_y, color = type_chr), width = 0, alpha = .6)+
  facet_grid(type_chr~subnum)+
  labs(x = "|Value Stim - Value Reference|", y = "Mean Log RT", title="Y/N Task (3 levels no shape)", alpha="", size = "")+
  theme(panel.grid.major.x  = element_blank(),
        panel.grid.minor.x  = element_blank(),
        panel.grid.minor.y  = element_blank(),
        legend.position = "bottom",
        axis.text.x = element_text(angle = 15, margin = margin(t = 5)),
        legend.box.margin=margin(t = -10),
        legend.spacing.y = unit(-0.25, "cm"),
        legend.box="vertical")+
  scale_color_manual(values = c("#1B9E77", "#7570B3", "#D95F02" ))+  
  # scale_y_continuous(breaks = seq(-1,1,.5), limits = c(-1.5, 1.5))+
  guides(linetype_chr = "none", color = "none")+
  scale_alpha_manual(values = c(1, .6, .5, .3, .2))+
  scale_size_manual(values = c(1.5, 2 , 2.5, 3, 3.5))

ggsave(file=paste0(fig_out_path, 'yn_subj_MeanLogRTValDiffAcrossTimeCHTnoshape.jpg'), p, height = 7, width = 12, units="in")
p

In [ ]:
k = 3

my_fwd_diff = matrix(c(2/k, -1/k, -1/k,
                       1/k, 1/k, -2/k), ncol = k-1)

my_fwd_diff

In [ ]:
fn_draws = paste0(inputs_path, 'yn_rtValDiff_chtnoshape_draws.csv')

if(file.exists(fn_draws)){
  yn_rt_draws = read.csv(fn_draws)
} else{
  
  normMax = 1
  normMin = 0
  
  yn_rt_draws = tibble()
  
  for(cur_sub in unique(data_yn_clean$subnum)){
  
    cur_dat = data_yn_cht_noshape %>% 
      ungroup() %>%
      filter(subnum == cur_sub) %>%
      mutate(type_chr = factor(type_chr, levels = c("RE", "CHT", "HT")),
             norm_day = (normMax - normMin) / (max(day) - min(day)) * (day - max(day)) + (normMax),
             log_rt = log(rt)) %>%
      group_by(day) %>% # Grouping by day to normalize valdiff matters. This makes sure every day ranges [-1,1]
      mutate(norm_absValDiff = (normMax - normMin) / (max(abs_val_diff) - min(abs_val_diff)) * (abs_val_diff - max(abs_val_diff)) + (normMax)) %>%
      ungroup() %>%
      select(log_rt, norm_absValDiff, norm_day, type_chr)
    
    contrasts(cur_dat$type_chr) = my_fwd_diff
    
    cur_m <- brm(formula = log_rt ~ norm_absValDiff * norm_day * type_chr,
                        data= cur_dat,
                        warmup = 500,
                        iter = 2500,
                        chains = 4,
                        init = "0",
                        cores = 4,
                        seed = 389253)
    
    cur_draws = as_draws_df(cur_m)
    cur_draws$subnum = cur_sub
    
    yn_rt_draws = rbind(yn_rt_draws, cur_draws)
  }
  names(yn_rt_draws) = gsub(":", ".", names(yn_rt_draws))
  write.csv(yn_rt_draws, fn_draws, row.names = F)

  rm(cur_dat, cur_m, cur_draws, cur_sub)
}


In [ ]:
yn_par_summary = as_tibble(yn_rt_draws) %>%
  select(b_Intercept, b_norm_absValDiff, b_norm_day, b_type_chr1, b_type_chr2, b_norm_absValDiff.norm_day, b_norm_absValDiff.type_chr1, b_norm_absValDiff.type_chr2, b_norm_day.type_chr1, b_norm_day.type_chr2, b_norm_absValDiff.norm_day.type_chr1, b_norm_absValDiff.norm_day.type_chr2, subnum) %>%
  gather(key, value, -subnum) %>%
  mutate(key=recode(key, 
                    "b_Intercept" = "Intercept", 
                    "b_norm_absValDiff"  = "|Value Diff| [0, 1]",
                    "b_norm_day" = "Day [0, 1]",
                    "b_type_chr1" = "Type (RE vs CHT)",
                    "b_type_chr2" = "Type (CHT vs HT)",
                    "b_norm_absValDiff.norm_day" = "|Value Diff| x Day",
                    "b_norm_absValDiff.type_chr1" = "|Value Diff| x Type (RE vs CHT)",
                    "b_norm_absValDiff.type_chr2" = "|Value Diff| x Type (CHT vs HT)",
                    "b_norm_day.type_chr1" = "Day x Type (RE vs CHT)",
                    "b_norm_day.type_chr2" = "Day x Type (CHT vs HT)",
                    "b_norm_absValDiff.norm_day.type_chr1" = "|Value Diff| x Day x Type (RE vs CHT)",
                    "b_norm_absValDiff.norm_day.type_chr2" = "|Value Diff| x Day x Type (CHT vs HT)",),
         key = factor(key, levels = c("Intercept", "|Value Diff| [0, 1]", "Day [0, 1]", "Type (RE vs CHT)", "Type (CHT vs HT)", "|Value Diff| x Day", "|Value Diff| x Type (RE vs CHT)", "|Value Diff| x Type (CHT vs HT)", "Day x Type (RE vs CHT)", "Day x Type (CHT vs HT)", "|Value Diff| x Day x Type (RE vs CHT)", "|Value Diff| x Day x Type (CHT vs HT)"))) %>%
  group_by(subnum, key) %>%
  summarise(.groups = 'keep',
            m = mean(value),
            l95 = HDInterval::hdi(value)[1],
            h95 = HDInterval::hdi(value)[2]) %>%
  mutate(subnum = as.factor(subnum),
         task = "Y/N") 

In [ ]:
p = yn_par_summary %>%
  ggplot(aes(m, subnum)) +
  geom_vline(aes(xintercept = 0), color = "gray") + 
  geom_point(position = position_dodge(width=.75))+
  geom_errorbarh(aes(xmin = l95, xmax = h95, height=.2), position = position_dodge(width=.75))+
  facet_grid(.~key, scales = 'free_x', labeller = labeller(key = label_wrap_gen(10))) +
  labs(x = "Parameter Estimate", y = "Subject", color = "", title = "95% HDI for posterior slopes - log(RT) ~ Value Diff x Day x Type (3 levels; no shape)") +
  theme(legend.position = "bottom",
        panel.grid = element_blank(),
        legend.box.margin=margin(t = -10))+
  scale_x_symmetric()

# ggsave(file=paste0(fig_out_path, 'yn_rtvalDiff_chtnoshape_ParEsts.jpg'), p, height = 3, width=12, units="in")

p